**Method using temperature matching between scene and calibration one set.**

In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset_location = '../dataset/' # put the root or relative path to your dataset here

In [3]:
scene_df = pd.read_hdf(dataset_location + 'scene.hdf5')
calibration_df = pd.read_hdf(dataset_location + 'calibration_set_1.hdf5')

In [4]:
# Function to find the closest calibration image
def find_closest_calibration_image(scene_temp, calibration_data):
    calibration_temperatures = calibration_data['t_fpa'].values
    closest_index = (np.abs(calibration_temperatures - scene_temp)).argmin()
    return calibration_data.iloc[closest_index]['image']

# Function to apply denoising using the closest calibration image
def denoise_image(scene_image, calibration_image):
    return scene_image - calibration_image

# Apply denoising to all scene images
corrected_scene_images = []
for i, row in scene_df.iterrows():
    scene_temp = row['t_fpa']
    scene_image = row['image']
    closest_calibration_image = find_closest_calibration_image(scene_temp, calibration_df)
    corrected_image = denoise_image(scene_image, closest_calibration_image)
    corrected_scene_images.append(corrected_image)

# Convert corrected_scene_images to numpy array
corrected_scene_images = np.array(corrected_scene_images)

In [6]:
def save_video_from_images(images, output_filename, fps=60):
    """
    Save a sequence of images as a video.

    Parameters:
    images (list or array): List or array of images (each image should be a 2D or 3D NumPy array).
    output_filename (str): Output video filename (e.g., 'output.avi').
    fps (int): Frames per second for the video.
    """
    # Get the dimensions of the images
    height, width = images[0].shape[:2]

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

    for img in images:
        # Ensure image is in the correct format (uint8)
        img_uint8 = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        # If the image is grayscale, convert it to BGR
        if len(img_uint8.shape) == 2:
            img_uint8 = cv2.cvtColor(img_uint8, cv2.COLOR_GRAY2BGR)
        out.write(img_uint8)

    # Release everything when the job is finished
    out.release()

output_filename = 'output-matching.avi'
save_video_from_images(corrected_scene_images, output_filename)